In [17]:
import cv2
import numpy as np

In [1]:
def adjust_brightness(image, factor):
    """Adjust image brightness by scaling the V channel in HSV."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = np.clip(v * factor, 0, 255).astype(np.uint8)
    hsv = cv2.merge([h, s, v])
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [3]:

# Extended HSV ranges for multiple colors
HSV_RANGES = {
    "blue":   ([100, 150, 50], [140, 255, 255]),
    "green":  ([40, 70, 70],  [80, 255, 255]),
    "yellow": ([20, 100, 100], [30, 255, 255]),
    "red1":   ([0, 120, 70],   [10, 255, 255]),
    "red2":   ([170, 120, 70], [180, 255, 255]),
    "orange": ([10, 100, 20],  [25, 255, 255]),
    "purple": ([130, 50, 50],  [160, 255, 255]),
    "pink":   ([160, 50, 50],  [170, 255, 255]),
    "white":  ([0, 0, 200],    [180, 25, 255]),
    "black":  ([0, 0, 0],      [180, 255, 30]),
}

In [5]:
# BGR colors for drawing bounding boxes
BGR_COLORS = {
    "blue": (255, 0, 0),
    "green": (0, 255, 0),
    "yellow": (0, 255, 255),
    "red": (0, 0, 255),
    "orange": (0, 165, 255),
    "purple": (255, 0, 255),
    "pink": (203, 192, 255),
    "white": (255, 255, 255),
    "black": (0, 0, 0),
}


In [7]:
def detect_colors(image, color_names=None, min_area=500):
    """Detect specified colors in the image and return bounding boxes."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    detections = []
    names = color_names or HSV_RANGES.keys()

    for name in names:
        if name not in HSV_RANGES:
            continue
        lower, upper = HSV_RANGES[name]
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        # noise removal
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            if cv2.contourArea(cnt) > min_area:
                x, y, w, h = cv2.boundingRect(cnt)
                # unify red ranges
                base_name = "red" if name.startswith('red') else name
                detections.append((base_name, (x, y, x + w, y + h)))
    return detections

In [9]:
def create_controls():
    """Create trackbars for interactive parameter tuning."""
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)
    cv2.createTrackbar('Brightness x10', 'Controls', 12, 30, lambda x: None)
    cv2.createTrackbar('Min Area', 'Controls', 500, 5000, lambda x: None)

In [11]:

def get_control_values():
    """Fetch current values from controls."""
    b = cv2.getTrackbarPos('Brightness x10', 'Controls') / 10.0
    area = cv2.getTrackbarPos('Min Area', 'Controls')
    return max(b, 0.1), max(area, 100)

In [13]:

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open camera")
        return

    create_controls()
    fps_counter = cv2.getTickCount()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        flip = cv2.flip(frame, 1)
        brightness_factor, min_area = get_control_values()
        bright = adjust_brightness(flip, brightness_factor)
        detected = detect_colors(bright, min_area=min_area)

        # draw detections
        for name, (x1, y1, x2, y2) in detected:
            color = BGR_COLORS.get(name, (0, 255, 0))
            cv2.rectangle(bright, (x1, y1), (x2, y2), color, 2)
            cv2.putText(bright, name.capitalize(), (x1, y2 + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # FPS calculation
        current_tick = cv2.getTickCount()
        fps = cv2.getTickFrequency() / (current_tick - fps_counter)
        fps_counter = current_tick
        cv2.putText(bright, f"FPS: {fps:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

        cv2.imshow("Color Detection Live", bright)
        key = cv2.waitKey(1) & 0xFF
        if key in (27, ord('q')):  # ESC or 'q'
            break

    cap.release()
    cv2.destroyAllWindows()

In [19]:
if __name__ == "__main__":
    main()